

# Solver for Problem 2 (Maximum Connectivity Clustering Model - G=(V,E))

### Ítalo Gomes Santana, Rafael Azevedo e Rodrigo Laigner

### Pontifical Catholic University of Rio de Janeiro (PUC-RIO) 2018.2


In [ ]:
"""
@author: ramscrz, rodrigo, italo
"""

import numpy as np

import sys
import math
import random
import string
import time as tm
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from gurobipy import *




### Loading and Initialization of the Input Graph for Problem 1 (Clique Clustering Model - G=(V,E)) 


In [ ]:
# Read Instance
def read_graph(file_name):
    
    print("\nLoading instance from file: ", file_name)
    f = open(file_name,'r+')
    
    vmap = {}
    graph = nx.Graph()
    
    for line in f.readlines():
        
        # Ignore comment lines and blocks in the file.
        if line.startswith("#"):
            continue

        # Read a line containing two integers, representing the two vertices of an edge.
        item_data = line.split()
        if len(item_data) != 2:
            # Invalid line data: 
            continue

        # Get the first vertex of the edge
        i = int(item_data[0])
        # Get the second vertex of the edge
        j = int(item_data[1])

        # Map each read vertex to a position of the vmap array, setting a unique integer to each vertex.
        if i not in vmap:
            vmap[i] = len(vmap) + 1
        if j not in vmap:
            vmap[j] = len(vmap) + 1

        # Set the source vertex index of the edge as the one mapped to the minimum integer.
        # Set the sink vertex index of the edge as the one mapped to the maximum integer.
        source = min(vmap[i], vmap[j])
        sink   = max(vmap[i], vmap[j])
        
        # Compute edges ignoring loops because distance is 0.
        # Edges weights are set as 1 by default.
        if source != sink and (source,sink) not in graph.edges(): 
            graph.add_edge(source, sink, weight=1)

    #print(vmap)
    f.close()
    
    V = { index:vertex for vertex,index in vmap.items() }
    
    print("N vértices: ", len(V))
    print("M edges: ", len(graph.edges()))
    
    # FOR DEBUG PURPOSES ONLY.
    #print("\nVertices: \n {}\n".format(graph.nodes()))
    #print("\nEdges: \n {}\n".format(graph.edges()))
    #print("\nEdges with weights: \n {}\n".format(graph.edges(data='weight')))
    #print(vmap.keys())
    
    return len(V), len(graph.edges()), vmap, graph, V

In [ ]:
def compute_distances(V,E):
    n = len(V)
    adjs = {}
    
    for v in V:
        adjs[v] = []
    
    # Compute the edge adjacency list for each vertex.
    for e in E:
        adjs[e[0]].append(e[1])
        adjs[e[1]].append(e[0])

    n = len(V)
    dist = {}

    # BFS to compute the minimum distance between each edge (v, s) for all vertices v, s of V. 
    for v in V:
        L=[v]
        dist[v,v]=0
        # While the frontier is not empty, continue BFS.
        while( len(L) > 0 ):
            c = L[0]
            L.remove(c)
            for s in adjs[c]:
                if ( (v,s) not in dist ):
                    dist[v,s] = dist[v,c] + 1
                    L.append(s)
    return dist

In [ ]:
# Basically a reverse map of vertices of an edge.
def getEdgeMapping(source, sink, vmap):
    return (vmap[source],vmap[sink])

# Get the edge vertices integer indexed by a unique integer.
# Basically a reverse map of vertices of an edge for every edge in E.
def remap_edges(E,V):
    return  [ (V[i],V[j]) for i,j in E]

In [ ]:
def visualizeGraph(graph, layout_attr='circular', color_map=None, scale=10, enable_edges=True, node_size=30):
    
    layout = None
    
    if(layout_attr=="bipartite"):
        # Position nodes in two straight lines.
        print("\nBipartite Graph: ")
        layout = nx.bipartite_layout(graph, scale=scale)
    elif(layout_attr=="circular"):
        # Position nodes on a circle.
        print("\Circular Graph: ")
        layout = nx.circular_layout(graph, scale=scale)
    elif(layout_attr=="kamada_kawai"):
        # Position nodes using Kamada-Kawai path-length cost-function.
        print("\Kamada-kawai Graph: ")
        layout = nx.kamada_kawai_layout(graph, scale=scale)
    elif(layout_attr=="random"):
        # Position nodes uniformly at random in the unit square.
        print("\Random Graph: ")
        layout = nx.random_layout(graph)
    elif(layout_attr=="rescale"):
        # Return scaled position array to (-scale, scale) in all axes.
        print("\Rescale Graph: ")
        layout = nx.rescale_layout(graph)
    elif(layout_attr=="shell"):
        # Position nodes in concentric circles.
        print("\Shell Graph: ")
        layout = nx.shell_layout(graph, scale=scale)
    elif(layout_attr=="spring"):
        #Position nodes using Fruchterman-Reingold force-directed algorithm.
        print("\Spring Graph: ")
        layout = nx.spring_layout(graph, scale=scale)
    elif(layout_attr=="spectral"):
        # Position nodes using the eigenvectors of the graph Laplacian.
        print("\nSpectral Graph: ")
        layout = nx.spectral_layout(graph, scale=scale)
    else:
        layout = nx.random_layout(graph, scale=scale)
    
    nx.draw_networkx_nodes(graph, layout)
    
    edge_labels = dict([((u,v,),d[ 'weight']) for u,v,d in graph.edges(data=True)])
    
    #nx.draw_networkx_labels(graph, layout, font_size=20, font_family='sans-serif')
    if(enable_edges):
        nx.draw_networkx_edges(graph, layout)
    plt.axis('off')
    
    plt.figure(3,figsize=(12,12))
    
    #nx.draw_networkx_edge_labels(graph, layout, edge_labels=edge_labels)
    
    if color_map != None:
        nx.draw(graph, layout, edge_cmap=plt.cm.Reds, node_size=node_size,font_size=8, node_color=color_map)
    else:
        nx.draw(graph, layout, edge_cmap=plt.cm.Reds, node_size=node_size,font_size=8)

    plt.show()

In [ ]:
def loadGraphInstance(file_name):
    n, m, vmap, graph, V = read_graph(file_name)
    dist = compute_distances(list(V.keys()), graph.edges())
    
    # FOR DEBUG PURPOSES ONLY.
    #print("VERTICES:\n")
    #print(V)
    #print("VMAP:\n")
    #print(vmap)
    #print("EDGES:\n")
    #print(graph.edges())

    E = remap_edges(graph.edges(), V)

    # FOR DEBUG PURPOSES ONLY
    #print("REVERSE MAPPED EDGES:\n")
    #print(E)
    #print("MINIMUM DISTANCE BETWEEN VERTICES:\n")
    #print(dist)
    
    return n, m, V, E, vmap, graph, dist

In [ ]:

# ######################################### FOR DEBUG PURPOSES ONLY #################################################

# n, m, V, E, vmap, graph, dist = loadGraphInstance("tvshow_edges_shorter.txt")

# print("\nEncoded N: ", n, "- Vertices:", graph.nodes())
# print("\nEncoded M: ", m, "- Edges:", graph.edges())

# print("\nN:", n, "- V:", V)
# print("\nM:", m, "- E:", E)

# scale = 1000
# en_edges = False

# print("\nCircular Layout: ")
# visualizeGraph(graph, layout_attr='circular', scale=scale, enable_edges=en_edges)
# print("\nKamada Kawai Layout: ")
# visualizeGraph(graph, layout_attr='kamada_kawai', scale=scale, enable_edges=en_edges)
# print("\nRandom Layout: ")
# visualizeGraph(graph, layout_attr='random', scale=scale, enable_edges=en_edges)
# print("\nShell Layout: ")
# visualizeGraph(graph, layout_attr='shell', scale=scale, enable_edges=en_edges)
# print("\nSpring Layout: ")
# visualizeGraph(graph, layout_attr='spring', scale=scale, enable_edges=en_edges)
# print("\nSpectral Layout: ")
# visualizeGraph(graph, layout_attr='spectral', scale=scale, enable_edges=en_edges)


## Formulação para o Problema 2 (Clique Clustering Model - G=(V,E))
## Clique Clustering Model - G=(V,E) - Fix # Clusters - Max conectividade

$$
{\large
\begin{array}{rll}
\min  &  \rho  &\\
s.t. & & \\
 & d(v,w).\delta_{vwk} \leq \rho & \quad \forall (v,w) \in V \times V \quad k = 1,\ldots,n\\
 & \delta_{vwk}\ =\ (y_{vk} * y_{wk})\ &\quad \forall (v,w) \in V \times V \quad k = 1,\ldots,n\\
 & y_{vk}\ \leq\ z_k & \quad \forall v \in V \quad \forall k= 1,\ldots, n \> = \> |V| \\
 & \sum\limits_{k=1}^{n} y_{vk} = 1 & \quad \forall v \in V \\
 &  \sum\limits_{k=1}^{n} z_k \leq \ell &\\
 &  y_{vk} \in \{ 0,1 \} &\quad \forall (v,k) \in V \times V\ where\ k = 1,\ldots,n & \\
 &  z_{k} \in \{ 0,1 \} &\quad k = 1,\ldots,n & \\
 &  \delta_{vw} \in \{ 0,1 \} &\quad \forall (v,w) \in V \times V & \\
\end{array}
}
$$






### Variables for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

# Create y_vk variables which represent the belonging of vertex v to the cluster k.
# If vertex v belongs to cluster k, then y_vk == 1. Otherwise, y_vk == 0.
# Binary decision variable.

def create_yvk_vars(model, V):
    n = len(V)
    
    y_vk = {}
    
    # For each vertex v and cluster k, vertex v belongs to cluster k (y_vk == 1) or not (y_vk == 0).
    for k in range(1, n+1):
        for v in range(1, n+1):
            y_vk[v, k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='y_vk'+'_'+str(v)+str(k))
    model.update()
    return y_vk

In [ ]:

# Create z_k variables which represent the inclusion of cluster k to the set of clusters composing the solution.
# If cluster k belongs to the set of disjoint partitions of V, then z_k == 1. Otherwise, z_k == 0.
# If z_k == 0, then no vertex is allocated to the cluster k and cluster k is excluded from the solution.

def create_zk_vars(model, V):

    z_k = {}
    
    for k in range(1, len(V) + 1):
        z_k[k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='z'+'_'+str(k))
    model.update()
    return z_k

In [ ]:

# Create delta_vwk binary variables which activates the diameter constraints dist(v, w) * d_vwk <= rho, where rho
# represents the maximum allowed diameter of a cluster and dist(v, w) the minimum distance between vertices v and w.
#
# Therefore:
#    1. if delta_vwk is 1, then vertices v and w belong to the same cluster, then dist(v, w) must be less or equal 
#       to rho.
#    2. if delta_vwk is 0, then vertices v and w don't belong to the same cluster, then dist(v, w) is multiplied by 
#       0 and this constraint doesn't regulate the dist(v, w).
#
# delta_vwk represents the status of belonging of vertices v and w to the cluster k. 

def create_d_vwk_vars(model, V, dist):

    n = len(V)
    d_vwk = {}
    
    for k in range(1, n + 1):
        for v in range(1, n):
            for w in range(v+1, n+1):
                if((v, w) in dist):
                    d_vwk[v, w, k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='delta_'+str(v)+"_"+str(w)+"_"+str(k))
    model.update()
    return d_vwk

In [ ]:

# Create rho variable which represent the maximum diameter allowed for any cluster. This diameter must be minimized.

def create_h_var(model, V):

    h = model.addVar(obj=1.0, vtype=GRB.INTEGER, name='rho_h')
    
    model.update()
    return h





### Constraints for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# Create diameter-limit constraints which restricts a pair of vertices (v, w) to belong to the same cluster k
# only if the minimum distance between these vertices is at most h (variable rho), the objective function variable.
# If the minimum distance between vertices v and w is greater than h (rho), then v or w belongs to cluster k
# or v, w not in cluster k.

def create_conf_vwk_constraints(model, y_vk, d_vwk, V, h, dist):

    n = len(V)
    total = 0
    print(n)
    conf_constrs = {}
    dist_h_constrs = {}
    y_vwk = {}
    print("create_conf_vwk_constraints")
    for k in range(1, n+1):
        for v in range(1, n):
            for w in range(v+1, n+1):
                total = total + 1
                if( (v, w, k) in d_vwk ):
                    total = total + 1
                    constr_name = 'conf_constr_'+str(v)+'_'+str(w)+'_'+str(k)
                    conf_constrs[v, w, k] = model.addConstr(d_vwk[v, w, k] == (y_vk[v, k] * y_vk[w, k]), name=constr_name)
                    constr_name = 'limit_distance_constr_'+str(v)+'_'+str(w)+'_'+str(k)
                    dist_h_constrs[v, w, k] = model.addConstr(dist[v, w] * d_vwk[v, w, k] <= h, name=constr_name)                    
                else:
                    constr_name_01 = 'conf_constr_else_'+str(v)+'_'+str(w)+'_'+str(k)
                    conf_constrs[v, w, k] = model.addConstr(y_vk[v, k] + y_vk[w, k] <= 1, name=constr_name_01)

    model.update()
    print("Total conflict constraints created = ", total)
    return conf_constrs, dist_h_constrs



In [ ]:

# Create upper bound constraints which assures that z_k will be assigned value 0 if a cluster holds no vertex.
# If there are more than 0 vertices in cluster k, then z_k might assume values 0 or 1 according to this constraint.
# However z_k must be 1 if exists more at least one vertex in cluster k. For this reason, the asgn_constraint is
# created in the next cells.

def create_up_constraints(model, y_vk, z_k, V):
    n = len(V)
    total = 0
    up_constrs = {}

    print("create_up_constraints")
    
    for k in range(1, n + 1):
        for v in range(1, n + 1):
            total = total + 1
            up_constrs[v,k] = model.addConstr(z_k[k] >= y_vk[v, k], name='up_constr_'+str(v)+'_'+str(k))
    
    model.update()
    print("Total up constraints created = ", total)
    return up_constrs

In [ ]:

# Create Assignment constraints which restricts a vertex to belong only to one cluster at a time.
# Each vertex belongs to one and one only cluster because partitions must be disjoint subsets of vertices.
# If exist a vertex v that belongs to cluster k, then z_k must be assigned value 1. 
# Otherwise, z_k must be assigned 0. This is assured by the quicksum of the product y_vk[v, k] * z_k[k].
    
def create_asgn_constraints(model, y_vk, V):
    n = len(V)
    total = 0
    asgn_constrs = {}
    
    for v in range(1, n + 1):
        total = total + 1
        constr_name = 'asgn_constr'+str(v)
        asgn_constrs[v] = model.addConstr((quicksum(y_vk[v, k] for k in range(1, n + 1)) == 1), name=constr_name)
    
    model.update()
    print("\nTotal assignment constraints created = ", total)
    return asgn_constrs

In [ ]:

# Create number of custers limit constraints which assures that there are only at most l clusters k (z_k == 1).
# If z_k == 1, then cluster k exists in the solution. Otherwise, cluster k does not exist in the solution.
# l is a given parameter.

def create_nclusters_limit_constraints(model, z_k, V, l, en_exact_nc=False):
    
    n = len(V)
    nclusters_limit_constrs = None
    constr_name = 'nclusters_limit_constr'
    
    if en_exact_nc:
        nclusters_limit_constrs = model.addConstr((quicksum(z_k[k] for k in range(1, n + 1)) == l), name=constr_name)
    else:
        nclusters_limit_constrs = model.addConstr((quicksum(z_k[k] for k in range(1, n + 1)) <= l), name=constr_name)
    
    model.update()
    print("\nTotal number clusters limit constraints created = 1")
    return nclusters_limit_constrs
    

In [ ]:

# Create objective function.

def setObjective(model, h):
    model.setObjective(h, GRB.MINIMIZE)
    model.update()






### Primal-Dual Solver for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# l => Maximum number of clusters
def solveMaxConnClusters(V, E, vmap, graph, dist, l, timeLimit, en_exact_nc=False):
     
    n = len(V)
    m = len(E)
    
    mp = Model()
    
    # Binary variables 
    y_vk = {}
    y_vwk = {}
    z_k = {}
    d_vw = {}
    d_vwk = {}
    
    
    # Constraints
    conf_constrs = {}
    dist_h_constrs = {}
    up_constrs = {}
    asgn_constrs = {}
    nclusters_limit_constrs = {}
    
    print("l (Maximum Number of Clusters) = ", l)
    
    vars_tstart = tm.time()
    
    print("Creating variables")
    
    # Create variables y_vk for each vertex v and cluster k
    # y_vk == 1 => vertex v belongs to cluster k
    # y_vk == 0 => vertex v does not belong to cluster k
    y_vk = create_yvk_vars(mp, V)
    
    print("yvk created")
    
    # Create variables z_k for each disjoint cluster k
    # z_k = 1 => cluster k exists.
    # z_k = 0 => cluster k does not exist.
    z_k = create_zk_vars(mp, V)
    
    print("zk created")
    
    # Create variables d_vw for each existing edge (v, w)
    # d_vw = 1 => dist(v, w) <= h (rho)
    # d_vw = 0 => dist(v, w) > h (rho).
    d_vwk = create_d_vwk_vars(mp, V, dist)
    
    print("d vwk created")
    
    # Create objective variable h which represents the minimum diameter of clusters for having l clusters
    h = create_h_var(mp, V)
    
    vars_exec_time = tm.time() - vars_tstart
    print("Variables created (execution time = {:.4f}s).".format(vars_exec_time))
    print("Creating constraints")
    
    constrs_tstart = tm.time()
    
    # Create constraint for avoiding conflict (vertices farther than H + 1 belong to different cluster).
    conf_constrs, dist_h_constrs = create_conf_vwk_constraints(mp, y_vk, d_vwk, V, h, dist)
    
    print("conf vwk created")
    
    # Create upper bound constraints (y_vk <= z_k)
    up_constrs = create_up_constraints(mp, y_vk, z_k, V)
    
    print("up created")
    
    # Create assignment constraint (each vertex belongs to one and one only cluster)
    asgn_constrs = create_asgn_constraints(mp, y_vk, V)
    
    print("asgn created")
    
    # Create number of clusters limit constraint (there can't be more than l clusters)
    nclusters_limit_constrs = create_nclusters_limit_constraints(mp, z_k, V, l, en_exact_nc)
    
    print("nclusters created")
    
    setObjective(mp, h)
    print("setobj")
    constrs_exec_time = tm.time() - constrs_tstart
    print("Constraints created (execution time = {:.4f}s).\n".format(constrs_exec_time))
    
    # Time limit for searching an optimal solution.

    print("TimeLimit: {}\n".format(timeLimit))
    
    mp.write("mf_clust.lp")
    mp.setParam('TimeLimit', timeLimit)
    mp.setParam('OutputFlag', 1)
    
    print()
    print("\n####### SOLVER START #######\n")
    solver_tstart = tm.time()
    mp.optimize()
    solver_exec_time = tm.time() - solver_tstart
    print("\nSolver finished (execution time = {:.4f}s)\n".format(solver_exec_time))
    print("\n####### SOLVER END #######\n")
    print()
    
    zp = mp.getObjective()

    
    if(mp.getAttr(GRB.Attr.ModelSense) == 1):
        print("Minimization")
    else:
        print("Maximization")
    print("Objective function: {}".format(zp.getValue()))
    print("TOTAL EXECUTION TIME = {:.4f}s\n".format(vars_exec_time + constrs_exec_time + solver_exec_time))
    
    #v_sol = mp.getAttr('X', y_vk)
    #print v_sol
    
    # Retrieve for each vertex the respective cluster to which it belongs to.
    #v_sol_r = {v:k for v in range(1, n + 1) for k in range(1, n + 1) if v_sol[v,k] > 0.001}
    
    #clusters = {}
        
    #for v,k in v_sol_r.items():
    #    if k not in clusters.keys():
    #        clusters[k] = []
    #    clusters[k].append(v)
    
    mp.write("mp.lp")
    dvw_sol = mp.getAttr('X', d_vwk)
    yvk_sol = mp.getAttr('X', y_vk)
    zk_sol = mp.getAttr('X', z_k)
    yvk = {v:k for v in range(1, n + 1) for k in range(1, n + 1) if yvk_sol[v,k] > 0.001}
    #print(zk_sol)
    #print(yvk)
    #print(yvk_sol)
    #print(dvw_sol)
    
    clusters = {}
    
    for v, w, k in dvw_sol.keys():
        if( dvw_sol[v, w, k] == 1 ):
            yvk_c = yvk[v]
            ywk_c = yvk[w]
            #print("d_vwk( v: {}, w: {}, k: {} ) = {}".format(v, w, k, dvw_sol[v, w, k]))
            #print("DISTANCE [ v: {}, w: {} ] = {}".format(v, w, dist[v, w]))
            if yvk_c not in clusters.keys():
                clusters[yvk_c] = []
            if v not in clusters[yvk_c]:
                clusters[yvk_c].append(v)
            if ywk_c not in clusters.keys():
                clusters[ywk_c] = []
            if w not in clusters[ywk_c]:
                clusters[ywk_c].append(w)
            if yvk_c != ywk_c:
                print("ERROR: d_vwk( v: {}, w: {}, k: {} ) == 1 but v in cluster {} and w in cluster {}".format(v, w, k, yvk_c, ywk_c))
    
    #for k in clusters.keys():
    #    print("Cluster #", k)
    #    print("\t{}".format(clusters[k]))
    
    return zp.getValue(), yvk, clusters






### Output of Results for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# Create a mapping of color shades for each cluster and set the corresponding cluster color to each vertex.

def computeColorMap(graph, clusters, vertex_cluster):
    assert(isinstance(clusters, dict))
    assert(isinstance(vertex_cluster, dict))
    
    if(not clusters or not vertex_cluster):
        return None
    
    color_map = []
    
    cmap = cm.autumn
    norm = Normalize(vmin=0,vmax=1)

    ratio = 1.0 / len(clusters)
    k_index = {}
    k_counter = 1
    for node in graph:
        k = vertex_cluster[node]
        #print("\tNode: {} => k = {}".format(node, k))
        if k not in k_index.keys():
            k_index[k] = k_counter
            k_counter += 1
        color_map.append(cmap(norm(k_index[k] * ratio)))
    
    return color_map


In [ ]:
def printSolution(zp, vertex_cluster, clusters, V, log_file_name=None):
    assert(isinstance(vertex_cluster, dict))
    assert(isinstance(clusters, dict))
    
    text_file = None
    
    if(isinstance(log_file_name, str)):
        text_file = open(log_file_name, "a")
        text_file.write("\nMinimum Maximum Distance in Clusters: {}\n".format(zp))
    print ("\nMinimum Maximum Distance in Clusters: ", zp)
    
    if(vertex_cluster != None and vertex_cluster):
        print ("\nSolution (vertex_index, k_cluster_index): \n\n\t{}\n".format(vertex_cluster))
        if(text_file != None):
            text_file.write("\nSolution (vertex_index, k_cluster_index): \n\n\t{}\n".format(vertex_cluster))
    
    if(isinstance(log_file_name, str)):
        text_file = open(log_file_name, "a")
        text_file.write("\n\n====> Clusters:\n\n")
    if(clusters != None and clusters):
        for k,vertices in clusters.items():
            print("\tCluster #{} contains the following vertices: \n".format(k))
            if(text_file != None):
                text_file.write("\tCluster #{} contains the following vertices: \n".format(k))
            line_buffer = 0
            for v in vertices:
                if(line_buffer == 0):
                    print("\t\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]), end = '')
                    if(text_file != None):
                        text_file.write("\t\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]))
                else:
                    print("\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]), end = '')
                    if(text_file != None):
                        text_file.write("\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]))
                line_buffer += 1
                
                # Blank line paragraph after printing 3 vertices.
                if(line_buffer == 3):
                    print()
                    if(text_file != None):
                        text_file.write("\n")
                    line_buffer = 0
            print("\n")
            if(text_file != None):
                text_file.write("\n")
    
    if(text_file != None):
        text_file.close()





### Execution of Primal-Dual Algorithm for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

def solveProbClusterMaxConnect(file_path, list_L, en_exact_nc=False, timeLimit=5000, show_graph=False, log_file_name=None):
    
    ###################################
    # file_path = "t01.txt"
    ###################################
    
    ###################################
    # Maximum number of clusters
    # list_L = [2, 1, 3]
    ###################################
    
    ###################################
    # If True, solves to find exactly l clusters. Otherwise, solves for any number of clusters less than l.
    # en_exact_nc = False
    ###################################

    ###################################
    # timeLimit = 60*60*4
    ###################################
    
    n, m, V, E, vmap, graph, dist = loadGraphInstance(file_path)
    
    print("\n\nInput file: {}\n".format(file_path))
    
    # Iterate over all maximum number of clusters provided. The problem is separately solved for each l.
    for l in list_L:
        
        print("\n=====================================================================\n")
        print("++++++++++++++++++++++++ ITERATION FOR L = {} ++++++++++++++++++++++++".format(l))
        print("\n=====================================================================\n")
        print("l = ", l)
        
        opt_val = None
        vertex_cluster = {}
        clusters = {}

        # Solve the problem of minimizing the diameter in clusters given a maximum number of cluster equal to l.
        opt_val, vertex_cluster, clusters = solveMaxConnClusters(V, E, vmap, graph, dist, l, timeLimit, en_exact_nc)
    
        if(opt_val != None):
            print("Opt MIN maximum distance between vertices in clusters = ", opt_val)
    
        ###################################
        #for v, w in dist:
            #print("DISTANCE [ v: {}, w: {} ] = {}".format(v, w, dist[v, w]))
        ###################################
    
        log_file_n = None
        log_file = None
        randomUID = None
        
        # Writes the header of the LOG FILE if a log file name has been provided.
        if(isinstance(log_file_name, str)):
            randomUID = ''.join(random.choice(string.ascii_lowercase) for i in range(10))
            log_file_n = log_file_name + "_l=" + str(l) + "_" + randomUID + ".txt"
            print("\nLOG FILE: {}\n".format(log_file_n))
            log_file = open(log_file_n, "a")
            log_file.write("\n=====> LOG FILE FOR PROBLEM 02 MAX CONNECTIVITY CLUSTERING\n")
            log_file.write("\n\nInput file: {}\n".format(file_path))
            log_file.write("\nITERATION FOR L = {} \n".format(l))
            log_file.close()
            log_file = None
        
        # Prints the found solution (also writes the solution to the log file if provided).
        if(opt_val!=None and vertex_cluster!=None and clusters!=None):
            if(log_file_n != None):
                printSolution(opt_val, vertex_cluster, clusters, V, log_file_n)
            else:
                printSolution(opt_val, vertex_cluster, clusters, V)

            if show_graph:
                color_map = computeColorMap(graph, clusters, vertex_cluster)

                #print("\nColor Map: \n")
                #print("\t ", color_map)

                visualizeGraph(graph, layout_attr='random', color_map=color_map, scale=1000, enable_edges=True, node_size=50)
        else:
            if(isinstance(log_file_name, str)):
                log_file_n = log_file_name + "_l=" + str(l) + "_" + randomUID + ".txt"
                log_file = open(log_file_n, "a")
                log_file.write("Model is NOT FEASIBLE or NOT OPTIMAL.\n\n")
                log_file.close()
                log_file = None
            
            



### Execute the cell below to use the Primal-Dual algorithm for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

# Execute this cell to use the algorithm and find optimal solutions for the formulation above.

# Path of the file containing the edges of the graph.
file_paths = ["instances/as19990829.txt","instances/as19981229.txt","instances/as19990829.txt"]


# Path of the file containing the log of the algorithm (results). If NONE, then no log file is generated.
#log_file_path = "log_file_problem02"
log_file_path = None
# Maximum number of clusters having the minimum maximum diameter possible.
L = [2, 16, 67]

timeLimit = 60*60*6

# Set the number of clusters to be exact if True. Otherwise, the number of cluster must be equal or less than l.
en_exact_nc = True

# Solve for each input file path in file_paths.
# for file_path in file_paths:
#     solveProbClusterMaxConnect(file_path=file_path, list_L=L, en_exact_nc=en_exact_nc, timeLimit=timeLimit,
#                                show_graph=True, log_file_name=log_file_path)
 


# Solve for each input file path in file_paths.
for file_path in file_paths:
    solveProbClusterMaxConnect(file_path=file_path, list_L=L, en_exact_nc=en_exact_nc, timeLimit=timeLimit,
                               show_graph=True, log_file_name=log_file_path)
    

## Solver for Connectivity Maximization Problem (02) using Column Generation

## ColGen Formulation - Master Problem

$$
{\large
\begin{array}{rll}
\min  &  \sum\limits_{p \in \cal{P}} \lambda_p &\\
s.t. & & \\
(\mbox{Dual } \pi_v) & \sum\limits_{p \in \cal{P}} a_{vp} . \lambda_p = 1 & \quad \forall v \in V \\
(\mbox{Dual } \omega) & \sum\limits_{p \in \cal{P}} \lambda_p \leq \ell & \\
 &  \lambda_{p} \in \{ 0,1 \} &
\end{array}
}
$$

## ColGen Subproblem

> Column reduced cost: $\overline{c}_p = 1 - \omega - \sum\limits_{v \in V} \pi_v . a_v$

$$
{\large
\begin{array}{rll}
\max  &  \sum\limits_{v \in V} \pi_v . a_v &\\
s.t. & & \\
 & a_{v} + a_{w} \leq 1  &\quad  d(v,w) > H + 1 \\
 &  a_{v} \in \{ 0,1 \} & \quad \forall v \in V
\end{array}
}
$$





## Variables for Problem 2 using Column Generation (Clique Clustering Model - G=(V,E)) 




### SUB-PROBLEM Variables 


In [ ]:
# Read Instance

def graph_read(file_name):
    global n, m, arc_i, arc_j, Adj, dist

    f = open(file_name,'r+')

    arc_i = {}
    arc_j = {}
    
    infty_cost = 1.0

    vertexSet = []
    row = 0
    for line in f.readlines():
        item_data = line.split()
        if (row == 0):
            n = int(item_data[0])
            print("Vertices",n)

        if (row == 1):
            m = int(item_data[0])
            print("Arestas",m)
            
        if (row > 1):
            i = row - 1
            x = int(item_data[0])
            arc_i[i] = x
            if x not in vertexSet:
                vertexSet.append(x)
            x = int(item_data[1])
            arc_j[i] = x
            if x not in vertexSet:
                vertexSet.append(x)
            #print(i, arc_i[i], arc_j[i])
        row += 1
    f.close()

    vertexP = {}
    vertexN = {}
    vertexSet.sort()
    ind = 1
    for v in vertexSet:
        vertexP[ind] = v
        vertexN[v] = ind
        ind +=1
        
#     print(n)
#     print(ind)
    
    #print vertexSet
    #print "         ------ jhskjehf "
    #print vertexP
    #print "         ------ jhskjehf "
    #print vertexN

    listAdj = {}
    for i in range(1,n+1):
        listAdj[i] = []

    Adj = {}
    for i in range(1, n):
        for j in range(i+1, n+1):
            Adj[i,j] = 1

    for l in range(1, m+1):
        arc_i[l] = vertexN[arc_i[l]]
        arc_j[l] = vertexN[arc_j[l]]
        if arc_i[l] < arc_j[l]:
            Adj[arc_i[l], arc_j[l]] = 0
            listAdj[arc_i[l]].append(arc_j[l])
            listAdj[arc_j[l]].append(arc_i[l])
            
#    for i in range(1,n+1):
#        print 'List: ', i, listAdj[i]
    
    dist = {}
    for s in range(1,n+1):    
        for i in range(1,n+1):
            dist[s,i] = -1
        dist[s,s] = 0

        q = []
        q.append(s)
        
        while len(q)>0:
            v = q[0]
            q.remove(q[0])
            for w in listAdj[v]:
                if dist[s,w] == -1:
                    dist[s,w] = dist[s,v] + 1
                    q.append(w)
                    
#    for s in range(1,n+1):
#        print s
#        for v in range(1,n+1):
#            print s,'->', v, ' : ',dist[s,v] 




In [ ]:
# Create y_v variables which represent the belonging of vertex v to a cluster p.
# If vertex v belongs to cluster a, then y_v == 1. Otherwise, y_v == 0.
# Binary decision variable.

def create_yv_vars_sub(model, yv):
    global n, m, arc_i, arc_j, Adj
    
    for v in range(1, n+1):
        yv[v] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='y'+'_'+str(v))
    model.update()




### MASTER-PROBLEM Variables 


In [ ]:
# Create lamb_p variables which represent the belonging of cluster p to the subset of clusters that is minimum and.
# a feasible solution to the problem of maximum connectivity.If cluster p belongs to the solution, then lamb_v == 1. 
# Otherwise, lamb_p == 0.
# Decision variable.

def create_lamb_vars_master(model, lamb):
    global n, m, arc_i, arc_j, Adj
    
    for p in range(1, n+1):
        lamb[p] = model.addVar(obj=1.0, vtype=GRB.CONTINUOUS, name='lamb'+'_'+str(p))
    model.update()





## Constraints for Problem 2 using Column Generation (Clique Clustering Model - G=(V,E)) 




### SUB-PROBLEM Constraints 


In [ ]:
# ColGen Subprob Create Conflict Constraints

def create_conf_sub_constr_sub(model, conf, yv):
    global n, m, arc_i, arc_j, Adj, dist, H
    print("H: ",H)
    for i in range(1, n):
        for j in range(i+1, n+1):
            if dist[i,j] > H + 1:
                conf[i,j] = model.addConstr(yv[i] + yv[j] <= 1, name='conf_'+str(i)+'_'+str(j))                                            
    
    model.update()




### MASTER-PROBLEM Constraints 


In [ ]:
# ColGen Master Set Part Constraints

def create_sp_constr_master(model, part, lamb, v_lamb):
    global n, m, arc_i, arc_j, Adj

    
    for v in range(1, n+1):
        v_lamb[v] = [v]
        part[v] = model.addConstr(lamb[v] == 1, name='part_'+str(v))                                            
    
    model.update()


In [ ]:
# ColGen Master Cardinality Constraints

def create_card_constr_master(model, lamb):
    global n, m, arc_i, arc_j, Adj, ell

    
    slack_card = model.addVar(obj=100.0, vtype=GRB.CONTINUOUS, name='slack')
    card = model.addConstr(quicksum(lamb[v] for v in range(1,n+1)) - slack_card <= ell, name='card')                                            
    
    model.update()
    
    return(card)




### Creation and Solver for the Sub-Problem


In [ ]:
def create_sub():
    global yv, pi_v, conf, msub
    yv = {}
    pi_v = {}
    conf = {}
    create_yv_vars_sub(msub, yv)
    create_conf_sub_constr_sub(msub, conf, yv)
    

In [ ]:
def solvesub():
    global yv, conf, msub, pi_v
    
    for v in range(1,n+1):
        yv[v].setAttr('Obj', pi_v[v])

    msub.optimize()
    zp = msub.getObjective()
    return zp.getValue()




### Solver for the Master Problem using the Column Generation described above


In [ ]:
# ColGen Solve Linear Relaxation

def solve_lp_master(timeLimit):
    global yv, pi_v, conf, msub
    
    Epsilon = 0.001
    master = Model()
    lamb = {}
    v_lamb = {}
    
    vars_tstart = tm.time()
    create_lamb_vars_master(master, lamb)
    print("Variable LAMBDA (Master) created (execution time = {:.4f}s).".format(tm.time() - vars_tstart))

    part = {}
    vars_tstart = tm.time()
    create_sp_constr_master(master, part, lamb, v_lamb)
    print("Constraint Set Partition (Master) created (execution time = {:.4f}s).".format(tm.time() - vars_tstart))

    vars_tstart = tm.time()
    card = create_card_constr_master(master, lamb)
    print("Constraint Card (Master) created (execution time = {:.4f}s).".format(tm.time() - vars_tstart))
    
    msub = Model()
    

    create_sub()
    print("Subproblem created (execution time = {:.4f}s).".format(tm.time() - vars_tstart))
    
    master.setParam('OutputFlag', 0)
    master.setParam('TimeLimit', timeLimit)
    msub.setParam('OutputFlag', 0)
    msub.setParam('TimeLimit', timeLimit)
    
    # print("TimeLimit: {}".format(timeLimit))
    n_columns = n
    not_opt = 1
    total_time = 0.0
    while not_opt > 0:
        vars_tstart = tm.time()
        if total_time > timeLimit:
            master.update()
            break
        
        not_opt = 0
        master.optimize()
        if master.status == GRB.INFEASIBLE:
            return "infeasible"
        
        inc = tm.time() - vars_tstart
        print("Time duration = {:.4f}s.".format(inc))
        total_time = total_time + inc
        
        if total_time > timeLimit:
            print("Time limit reached",total_time)
            break
        zd = master.getObjective()


        for v in range(1,n+1):
            pi_v[v] = -part[v].getAttr("Pi")
            
        omega = card.getAttr("Pi")

        redcost = solvesub()
        redcost = - redcost
        
        #msub.write("sub.lp")
        print ('N columns:', n_columns,"/Redcost:",redcost,"Master V:",zd.getValue())
        if redcost >= 1 - omega + Epsilon:
            not_opt = 1
        
            v_sol = msub.getAttr('X', yv)
            #print v_sol
            v_sol_r = [v for v in range(1,n+1) if v_sol[v] > 0.01]

            n_columns += 1
            lamb[n_columns] = master.addVar(obj=1.0, vtype=GRB.CONTINUOUS, name='lamb'+'_'+str(n_columns))
            master.update()

            v_lamb[n_columns] = []
            for v in v_sol_r:
                v_lamb[n_columns].append(v)
                master.chgCoeff(part[v],lamb[n_columns], 1.0)
            master.update()
            
        else:
            not_opt = 0

    master.write("mm.lp")
    print ('N columns:', n_columns)
    
    l_sol = master.getAttr('X', lamb)
    #print v_sol
    
    l_sol_r = [r for r in range(1,n_columns+1) if l_sol[r] > 0.001]
    print (l_sol_r)
    
    for r in l_sol_r:
        print (r, l_sol[r])
        col = master.getCol(lamb[r])
        print (col)
        print (v_lamb[r])
    
    
    zd = master.getObjective()
    print ("Obj function:",zd.getValue())
    print("execution total_time",total_time)
    print("execution total_time= {:.4f}s.".format(total_time))
    return total_time
    



### Execute the cell below to use the Column Generation algorithm for Problem 2


In [ ]:

instance_path = "instances/"

files = []
timeLimits = []
nClusters = [2, 16, 67]

hFoundForCluster = [5, 15, 57]

files.append(instance_path + "as19990829.txt") # 103-243
timeLimits.append(60*60*4) # 4 horas

# files.append(instance_path + "as19981229.txt") #493-1189
# timeLimits.append(60*60*6) # 6 horas

# files.append(instance_path + "as19981230.txt") #493-1189
# timeLimits.append(60*60*6) # 6 horas


# iterate over instances
for i in range(0,len(files)):
    # iterate over # of clusters

    for j in range(0,len(nClusters)):
        #H = startH
        H = hFoundForCluster[j]
        
        #only for entering while
        BEST_H = H + 1
        
        print("Instância:", files[i]," # de clusters:",nClusters[j])
        
        #remove from while loop
        ell = nClusters[j]
        graph_read(files[i])
        
        lp_result = 0
        remaining_time = timeLimits[i]
        
        firstTry = True
        
        while H != BEST_H:
            
            lp_result = solve_lp_master(remaining_time)
            print("lp_result:",lp_result)
            if lp_result == "infeasible":
                print("infeasible")
                
                if (firstTry):
                    print("Nao ha melhor H encontrado. Selecione um H inicial viável.")
                    break
                
                print("Melhor H encontrado por ora: ",BEST_H)
                #let's go up
                H = int((BEST_H + H) / 2)
                
            else:
                print("feasible")
                #let's go down
                BEST_H = H
                H = int(BEST_H / 2)
            
            firstTry = False
                
            remaining_time = remaining_time - lp_result
            print("Remaining Time:",remaining_time)
            if(remaining_time <= 0.0 or H <= 0):
                print("Melhor H encontrado: ",BEST_H)    
                break
            
            print("H was updated %d"% (H))
            #H = H - 1
        print("### END OF WHILE!!!! ###")
        
        #let's try last tentative with the last number, H == 0
        #lp_result = solve_lp(remaining_time)
        #if lp_result == "infeasible":
        #    print("infeasible")
        #else:
        #    BEST_H = H
        
        print("Melhor H encontrado: ",BEST_H) 
        

## Outras Formulações (Tentativas)

## Formulação Alternativa 01 para o Problema 2 (Clique Clustering Model - G=(V,E))
## Clique Clustering Model - G=(V,E) - Fix # Clusters - Max conectividade


$$
{\large
\begin{array}{rll}
\min  &  \rho  &\\
s.t. & & \\
 & d(v,w).\delta_{vw} \leq \rho & \quad \forall (v,w) \in V \times V \quad k = 1,\ldots,n\\
 & \sum\limits_{k=1}^{n} (y_{vk} *\ y_{wk})\ =\ \delta_{vw}\ &\quad \forall (v,w) \in V \times V\\
 & \sum\limits_{k=1}^{n} y_{vk}\ =\ 1 & \quad \forall v \in V \\
 & \sum\limits_{k=1}^{n} (y_{vk}\ *\ z_{k})\ =\ 1 & \quad \forall v \in V \\
 & \sum\limits_{v=1}^{n} y_{vk}\ \geq\ z_{k}\ & \quad \forall k = 1,\ldots,n\\
 &  \sum\limits_{k=1}^{n} z_k \leq \ell &\\
 &  y_{vk} \in \{ 0,1 \} &\quad \forall (v,k) \in V \times V\ where\ k = 1,\ldots,n & \\
 &  z_{k} \in \{ 0,1 \} &\quad k = 1,\ldots,n & \\
 &  \delta_{vw} \in \{ 0,1 \} &\quad \forall (v,w) \in V \times V & \\
\end{array}
}
$$






### Variables for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

# Create y_vk variables which represent the belonging of vertex v to the cluster k.
# If vertex v belongs to cluster k, then y_vk == 1. Otherwise, y_vk == 0.
# Binary decision variable.

def create_yvk_vars_01(model, V):
    n = len(V)
    
    y_vk = {}
    
    # For each vertex v and cluster k, vertex v belongs to cluster k (y_vk == 1) or not (y_vk == 0).
    for k in range(1, n+1):
        for v in range(1, n+1):
            y_vk[v, k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='y_vk'+'_'+str(v)+str(k))
    model.update()
    return y_vk

In [ ]:

# Create z_k variables which represent the inclusion of cluster k to the set of clusters composing the solution.
# If cluster k belongs to the set of disjoint partitions of V, then z_k == 1. Otherwise, z_k == 0.
# If z_k == 0, then no vertex is allocated to the cluster k and cluster k is excluded from the solution.

def create_zk_vars_01(model, V):

    z_k = {}
    
    for k in range(1, len(V) + 1):
        z_k[k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='z'+'_'+str(k))
    model.update()
    return z_k

In [ ]:

# Create delta_vw binary variables which activates the diameter constraints dist(v, w) * d_vw <= rho, where rho
# represents the maximum allowed diameter of a cluster and dist(v, w) the minimum distance between vertices v and w.
#
# Therefore:
#    1. if delta_vw is 1, then vertices v and w belong to the same cluster, then dist(v, w) must be less or equal 
#       to rho.
#    2. if delta_vw is 0, then vertices v and w don't belong to the same cluster, then dist(v, w) is multiplied by 
#       0 and this constraint doesn't regulate the dist(v, w).

def create_d_vw_vars_01(model, V, dist):

    n = len(V)
    d_vw = {}
    
    for v in range(1, n):
        for w in range(v+1, n + 1):
            if((v, w) in dist):
                d_vw[v, w] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='delta_'+str(v)+"_"+str(w))
    model.update()
    return d_vw

In [ ]:

# Create delta_vwk binary variables which activates the diameter constraints dist(v, w) * d_vwk <= rho, where rho
# represents the maximum allowed diameter of a cluster and dist(v, w) the minimum distance between vertices v and w.
#
# Therefore:
#    1. if delta_vwk is 1, then vertices v and w belong to the same cluster, then dist(v, w) must be less or equal 
#       to rho.
#    2. if delta_vwk is 0, then vertices v and w don't belong to the same cluster, then dist(v, w) is multiplied by 
#       0 and this constraint doesn't regulate the dist(v, w).
#
# delta_vwk represents the status of belonging of vertices v and w to the cluster k. 

def create_d_vwk_vars_01(model, V, dist):

    n = len(V)
    d_vwk = {}
    
    for k in range(1, n + 1):
        for v in range(1, n):
            for w in range(v+1, n+1):
                if((v, w) in dist):
                    d_vwk[v, w, k] = model.addVar(obj=0.0, vtype=GRB.BINARY, name='delta_'+str(v)+"_"+str(w)+"_"+str(k))
    model.update()
    return d_vwk

In [ ]:

# Create rho variable which represent the maximum diameter allowed for any cluster. This diameter must be minimized.

def create_h_var_01(model, V):

    h = model.addVar(obj=1.0, vtype=GRB.INTEGER, name='rho_h')
    
    model.update()
    return h





### Constraints for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# Create diameter-limit constraints which restricts a pair of vertices (v, w) to belong to the same cluster
# only if the minimum distance between these vertices is at most h (variable rho), the objective function variable.
# If the minimum distance between vertices v and w is greater than h (rho), then v or w belongs to cluster k
# or v, w not in cluster k.

def create_conf_vw_constraints_01(model, y_vk, d_vw, V, h, dist):

    n = len(V)
    total = 0
    
    conf_constrs = {}
    conf_v_constrs = {}
    conf_w_constrs = {}
    dist_h_constrs = {}

    for v in range(1, n):
        for w in range(v+1, n+1):
            if( (v, w) in d_vw ):
                total = total + 1
                constr_name = 'conf_constr_'+str(v)+'_'+str(w)
                conf_constrs[v, w] = model.addConstr(quicksum(y_vk[v, k] * y_vk[w, k] for k in range(1, n+1)) == d_vw[v, w], name=constr_name)
                constr_name = 'limit_distance_constr_'+str(v)+'_'+str(w)
                dist_h_constrs[v, w] = model.addConstr(dist[v, w] * d_vw[v, w] <= h, name=constr_name)
    
    model.update()
    print("\nTotal conflict constraints created = ", total)
    return conf_constrs, conf_v_constrs, conf_w_constrs, dist_h_constrs


In [ ]:

# Create upper bound constraints which assures that z_k will be assigned value 0 if a cluster holds no vertex.
# If there are more than 0 vertices in cluster k, then z_k might assume values 0 or 1 according to this constraint.
# However z_k must be 1 if exists more at least one vertex in cluster k. For this reason, the asgn_constraint is
# created in the next cells.

def create_up_constraints_01(model, y_vk, z_k, V):
    n = len(V)
    total = 0
    up_constrs = {}
    up_ref_constrs = {}

    for v in range(1, n + 1):
        total = total + 1
        up_constrs[v] = model.addConstr(quicksum(y_vk[v, k] * z_k[k] for k in range(1, n+1)) == 1, name='up_constr_'+str(v))

    for k in range(1, n + 1):
        total = total + 1
        up_ref_constrs[k] = model.addConstr(quicksum(y_vk[v, k] for v in range(1, n+1)) >= z_k[k], name='up_ref_constr_'+str(v)+'_'+str(k))
    
    model.update()
    print("\nTotal up constraints created = ", total)
    return up_constrs

In [ ]:

# Create Assignment constraints which restricts a vertex to belong only to one cluster at a time.
# Each vertex belongs to one and one only cluster because partitions must be disjoint subsets of vertices.
# If exist a vertex v that belongs to cluster k, then z_k must be assigned value 1. 
# Otherwise, z_k must be assigned 0. This is assured by the quicksum of the product y_vk[v, k] * z_k[k].
    
def create_asgn_constraints_01(model, y_vk, V):
    n = len(V)
    total = 0
    asgn_constrs = {}
    
    for v in range(1, n + 1):
        total = total + 1
        constr_name = 'asgn_constr'+str(v)
        asgn_constrs[v] = model.addConstr((quicksum(y_vk[v, k] for k in range(1, n + 1)) == 1), name=constr_name)
    
    model.update()
    print("\nTotal assignment constraints created = ", total)
    return asgn_constrs

In [ ]:

# Create number of custers limit constraints which assures that there are only at most l clusters k (z_k == 1).
# If z_k == 1, then cluster k exists in the solution. Otherwise, cluster k does not exist in the solution.
# l is a given parameter.

def create_nclusters_limit_constraints_01(model, z_k, V, l, en_exact_nc=False):
    
    n = len(V)
    nclusters_limit_constrs = None
    constr_name = 'nclusters_limit_constr'
    
    if en_exact_nc:
        nclusters_limit_constrs = model.addConstr((quicksum(z_k[k] for k in range(1, n + 1)) == l), name=constr_name)
    else:
        nclusters_limit_constrs = model.addConstr((quicksum(z_k[k] for k in range(1, n + 1)) <= l), name=constr_name)
    
    model.update()
    print("\nTotal number clusters limit constraints created = 1")
    return nclusters_limit_constrs
    

In [ ]:

# Create objective function.

def setObjective_01(model, h):
    model.setObjective(h, GRB.MINIMIZE)
    model.update()






### Primal-Dual Solver for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# l => Maximum number of clusters

def solveMaxConnClusters_01(V, E, vmap, graph, dist, l, timeLimit, en_exact_nc=False):
    
    n = len(V)
    m = len(E)
    
    mp = Model()
    
    # Binary variables 
    y_vk = {}
    y_vwk = {}
    z_k = {}
    d_vw = {}
    
    
    # Constraints
    conf_constrs = {}
    conf_constrs_00 = {}
    conf_v_constrs = {}
    conf_w_constrs = {}
    dist_h_constrs = {}
    up_constrs = {}
    asgn_constrs = {}
    nclusters_limit_constrs = {}
    
    print("\nl (Maximum Number of Clusters) = ", l)
    
    vars_tstart = tm.time()
    
    # Create variables y_vk for each vertex v and cluster k
    # y_vk == 1 => vertex v belongs to cluster k
    # y_vk == 0 => vertex v does not belong to cluster k
    y_vk = create_yvk_vars_01(mp, V)
    
    ### yvklb_constrs = create_yvklb_constraint(mp, V, y_vk)
    
    # Create variables z_k for each disjoint cluster k
    # z_k = 1 => cluster k exists.
    # z_k = 0 => cluster k does not exist.
    z_k = create_zk_vars_01(mp, V)
    
    # Create variables d_vw for each existing edge (v, w)
    # d_vw = 1 => dist(v, w) <= h (rho)
    # d_vw = 0 => dist(v, w) > h (rho).
    d_vw = create_d_vw_vars_01(mp, V, dist)
    
    # Create objective variable h which represents the minimum diameter of clusters for having l clusters
    h = create_h_var_01(mp, V)
    
    vars_exec_time = tm.time() - vars_tstart
    print("\nVariables created (execution time = {:.4f}s).".format(vars_exec_time))
    
    constrs_tstart = tm.time()
    #total = 0
    # Create constraint for avoiding conflict (vertices farther than H + 1 belong to different cluster).
    conf_constrs, conf_v_constrs, conf_w_constrs, dist_h_constrs = create_conf_vw_constraints_01(mp, y_vk, d_vw, V, h, dist)
    
    # Create upper bound constraints (y_vk <= z_k)
    up_constrs = create_up_constraints_01(mp, y_vk, z_k, V)
    
    # Create assignment constraint (each vertex belongs to one and one only cluster)
    asgn_constrs = create_asgn_constraints_01(mp, y_vk, V)
    
    # Create y_vk >= delta_vw for all (v, w) where v, w in V 
    # and delta_vw == 1 if and only if v and w belong to the same cluster.
    # ydelta_vwk_constrs = create_ydelta_vwk_constraints(mp, V, y_vk, d_vw)
    
    ### l_lowerbound_constr = create_l_constraint(mp, V, z_k)
    
    # Create number of clusters limit constraint (there can't be more than l clusters)
    nclusters_limit_constrs = create_nclusters_limit_constraints_01(mp, z_k, V, l, en_exact_nc)
    
    #print("TOTAL = ", total)
    
    setObjective_01(mp, h)
    
    constrs_exec_time = tm.time() - constrs_tstart
    print("\nConstraints created (execution time = {:.4f}s).\n".format(constrs_exec_time))
    
    # Time limit for searching an optimal solution.
    try: 
        print("\nTimeLimit: {}\n".format(timeLimit))

        mp.write("mf_clust.lp")
        mp.setParam('TimeLimit', timeLimit)
        mp.setParam('OutputFlag', 1)

        print()
        print("\n####### SOLVER START #######\n")
        solver_tstart = tm.time()
        mp.optimize()
        solver_exec_time = tm.time() - solver_tstart
        print("\nSolver finished (execution time = {:.4f}s)\n".format(solver_exec_time))
        print("\n####### SOLVER END #######\n")
        print()


        zp = mp.getObjective()

        if(mp.getAttr(GRB.Attr.ModelSense) == 1):
            print("Minimization")
        else:
            print("Maximization")
        print("Objective function: {}".format(zp))
        print("TOTAL EXECUTION TIME = {:.4f}s\n".format(vars_exec_time + constrs_exec_time + solver_exec_time))

        if mp.status == GRB.OPTIMAL:

            mp.write("mp.lp")
            dvw_sol = mp.getAttr('X', d_vw)
            yvk_sol = mp.getAttr('X', y_vk)
            zk_sol = mp.getAttr('X', z_k)
            yvk = {v:k for v in range(1, n + 1) for k in range(1, n + 1) if yvk_sol[v,k] > 0.001}

            # FOR DEBUG PURPOSES ONLY.
            #print(zk_sol)
            #print(yvk)
            #print()
            #print(yvk_sol)
            #print()
            #print(dvw_sol)
            #print()
            #print("{}\n\n".format(dist[2,4]))
            #print("{}\n\n".format(dist[2,5]))

            clusters = {}

            for v in yvk.keys():
                clstr_index = yvk[v]
                if clstr_index not in clusters.keys():
                    clusters[clstr_index] = []
                if v not in clusters[clstr_index]:
                    clusters[clstr_index].append(v)

            for v, w in dvw_sol.keys():
                if( dvw_sol[v, w] == 1 ):
                    #print("d_vw( v: {}, w: {} ) = {}".format(v, w, dvw_sol[v, w]))
                    #print("DISTANCE [ v: {}, w: {} ] = {}".format(v, w, dist[v, w]))
                    if yvk[v] != yvk[w]:
                        print("ERROR: d_vw( v: {}, w: {}) == 1 but v in cluster {} and w in cluster {}".format(v, w, yvk[v], yvk[w]))


            #nx.write_gpickle([zp.getValue(), yvk, clusters],"test.gpickle")

            return zp.getValue(), yvk, clusters

        else:

            print("Model is NOT FEASIBLE or NOT OPTIMAL: Status Code = {}\n\n".format(mp.status))

            return None, None, None
    except GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))

    except AttributeError:
        print('Encountered an attribute error')






### Output of Results for Problem 2 (Clique Clustering Model - G=(V,E))   >>>  IMPLEMENTATION


In [ ]:
# Create a mapping of color shades for each cluster and set the corresponding cluster color to each vertex.

def computeColorMap_01(graph, clusters, vertex_cluster):
    assert(isinstance(clusters, dict))
    assert(isinstance(vertex_cluster, dict))
    
    if(not clusters or not vertex_cluster):
        return None
    
    color_map = []
    
    cmap = cm.autumn
    norm = Normalize(vmin=0,vmax=1)

    ratio = 1.0 / len(clusters)
    k_index = {}
    k_counter = 1
    for node in graph:
        k = vertex_cluster[node]
        #print("\tNode: {} => k = {}".format(node, k))
        if k not in k_index.keys():
            k_index[k] = k_counter
            k_counter += 1
        color_map.append(cmap(norm(k_index[k] * ratio)))
    
    return color_map


In [ ]:
def printSolution_01(zp, vertex_cluster, clusters, V, log_file_name=None):
    assert(isinstance(vertex_cluster, dict))
    assert(isinstance(clusters, dict))
    
    text_file = None
    
    if(isinstance(log_file_name, str)):
        text_file = open(log_file_name, "a")
        text_file.write("\nMinimum Maximum Distance in Clusters: {}\n".format(zp))
    print ("\nMinimum Maximum Distance in Clusters: ", zp)
    
    if(vertex_cluster != None and vertex_cluster):
        print ("\nSolution (vertex_index, k_cluster_index): \n\n\t{}\n".format(vertex_cluster))
        if(text_file != None):
            text_file.write("\nSolution (vertex_index, k_cluster_index): \n\n\t{}\n".format(vertex_cluster))
    
    if(isinstance(log_file_name, str)):
        text_file = open(log_file_name, "a")
        text_file.write("\n\n====> Clusters:\n\n")
    if(clusters != None and clusters):
        for k,vertices in clusters.items():
            print("\tCluster #{} contains the following vertices: \n".format(k))
            if(text_file != None):
                text_file.write("\tCluster #{} contains the following vertices: \n".format(k))
            line_buffer = 0
            for v in vertices:
                if(line_buffer == 0):
                    print("\t\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]), end = '')
                    if(text_file != None):
                        text_file.write("\t\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]))
                else:
                    print("\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]), end = '')
                    if(text_file != None):
                        text_file.write("\tINDEX = {} ; vertex[{}] = {}".format(v, v, V[v]))
                line_buffer += 1
                
                # Blank line paragraph after printing 3 vertices.
                if(line_buffer == 3):
                    print()
                    if(text_file != None):
                        text_file.write("\n")
                    line_buffer = 0
            print("\n")
            if(text_file != None):
                text_file.write("\n")
    
    if(text_file != None):
        text_file.close()





### Execution of Primal-Dual Algorithm for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

def solveProbClusterMaxConnect_01(file_path, list_L, en_exact_nc=False, timeLimit=5000, show_graph=False, log_file_name=None):
    
    ###################################
    # file_path = "t01.txt"
    ###################################
    
    ###################################
    # Maximum number of clusters
    # list_L = [2, 1, 3]
    ###################################
    
    ###################################
    # If True, solves to find exactly l clusters. Otherwise, solves for any number of clusters less than l.
    # en_exact_nc = False
    ###################################

    ###################################
    # timeLimit = 60*60*4
    ###################################
    
    n, m, V, E, vmap, graph, dist = loadGraphInstance(file_path)
    
    print("\n\nInput file: {}\n".format(file_path))
    
    # Iterate over all maximum number of clusters provided. The problem is separately solved for each l.
    for l in list_L:
        
        print("\n=====================================================================\n")
        print("++++++++++++++++++++++++ ITERATION FOR L = {} ++++++++++++++++++++++++".format(l))
        print("\n=====================================================================\n")
        print("l = ", l)
        
        opt_val = None
        vertex_cluster = {}
        clusters = {}

        # Solve the problem of minimizing the diameter in clusters given a maximum number of cluster equal to l.
        opt_val, vertex_cluster, clusters = solveMaxConnClusters_01(V, E, vmap, graph, dist, l, timeLimit, en_exact_nc)
    
        if(opt_val != None):
            print("Opt MIN maximum distance between vertices in clusters = ", opt_val)
    
        ###################################
        #for v, w in dist:
            #print("DISTANCE [ v: {}, w: {} ] = {}".format(v, w, dist[v, w]))
        ###################################
    
        log_file_n = None
        log_file = None
        randomUID = None
        
        # Writes the header of the LOG FILE if a log file name has been provided.
        if(isinstance(log_file_name, str)):
            randomUID = ''.join(random.choice(string.ascii_lowercase) for i in range(10))
            log_file_n = log_file_name + "_l=" + str(l) + "_" + randomUID + ".txt"
            print("\nLOG FILE: {}\n".format(log_file_n))
            log_file = open(log_file_n, "a")
            log_file.write("\n=====> LOG FILE FOR PROBLEM 02 MAX CONNECTIVITY CLUSTERING\n")
            log_file.write("\n\nInput file: {}\n".format(file_path))
            log_file.write("\nITERATION FOR L = {} \n".format(l))
            log_file.close()
            log_file = None
        
        # Prints the found solution (also writes the solution to the log file if provided).
        if(opt_val!=None and vertex_cluster!=None and clusters!=None):
            if(log_file_n != None):
                printSolution_01(opt_val, vertex_cluster, clusters, V, log_file_n)
            else:
                printSolution_01(opt_val, vertex_cluster, clusters, V)

            if show_graph:
                color_map = computeColorMap_01(graph, clusters, vertex_cluster)

                #print("\nColor Map: \n")
                #print("\t ", color_map)

                visualizeGraph(graph, layout_attr='random', color_map=color_map, scale=1000, enable_edges=True, node_size=50)
        else:
            if(isinstance(log_file_name, str)):
                log_file_n = log_file_name + "_l=" + str(l) + "_" + randomUID + ".txt"
                log_file = open(log_file_n, "a")
                log_file.write("Model is NOT FEASIBLE or NOT OPTIMAL.\n\n")
                log_file.close()
                log_file = None
            
            



### Execute the cell below to use the Primal-Dual algorithm for Problem 2 (Clique Clustering Model - G=(V,E)) 


In [ ]:

# Execute this cell to use the algorithm and find optimal solutions for the formulation above.

# Path of the file containing the edges of the graph.
file_paths = ["instances/as19981229.txt"] 

# Path of the file containing the log of the algorithm (results). If NONE, then no log file is generated.
log_file_path = None #"log_file_problem02"

# Maximum number of clusters having the minimum maximum diameter possible.
L = [2, 16, 67]

timeLimit = 60*60*6

# Set the number of clusters to be exact if True. Otherwise, the number of cluster must be equal or less than l.
en_exact_nc = True

# Solve for each input file path in file_paths.
for file_path in file_paths:
    solveProbClusterMaxConnect(file_path=file_path, list_L=L, en_exact_nc=en_exact_nc, timeLimit=timeLimit,
                               show_graph=True, log_file_name=log_file_path)
    